In [5]:
from unittest.mock import Mock, patch, call

import brownie
import pytest
from unittest.mock import MagicMock
from brownie import multicall as brownie_multicall

from fastlane_bot import Bot, Config
from fastlane_bot.events.exchanges import UniswapV2, UniswapV3, SushiswapV2, CarbonV1, BancorV3
from fastlane_bot.events.managers.manager import Manager
Base = None
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC

print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV3))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(SushiswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonV1))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(BancorV3))
from fastlane_bot.testing import *

#plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)


ConstantProductCurve v2.14 (23/May/2023)
CarbonBot v3-b2.2 (20/June/2023)
UniswapV2 v0.0.1 (2023-07-03)
UniswapV3 v0.0.1 (2023-07-03)
SushiswapV2 v0.0.1 (2023-07-03)
CarbonV1 v0.0.1 (2023-07-03)
BancorV3 v0.0.1 (2023-07-03)
Version = 3-b2.2 [requirements >= 3.0 is met]


In [6]:
import json

with open("fastlane_bot/data/event_test_data.json", "r") as f:
    event_data = json.load(f)

with open("fastlane_bot/data/test_pool_data.json", "r") as f:
    pool_data = json.load(f)

In [7]:

# Create mock instances for all required parameters
cfg = Config.new(config=Config.CONFIG_TENDERLY)

# create manager instance for all tests
manager = Manager(cfg.w3, cfg, pool_data, 20, SUPPORTED_EXCHANGES=['bancor_v3', 'carbon_v1', 'uniswap_v2', 'uniswap_v3'])


## test_update_from_event_uniswap_v2

In [40]:
carbon_controller = manager.get_pool_contract('carbon_v1', '0xC537e898CD774e2dCBa3B14Ea6f34C93d5eA45e1')

carbon_controller.caller.pairTradingFeePPM(
    manager.cfg.USDC_ADDRESS, manager.cfg.LINK_ADDRESS
)

2000

In [45]:
carbon_controller.all_functions()

[<Function admin()>,
 <Function implementation()>,
 <Function upgradeTo(address)>,
 <Function upgradeToAndCall(address,bytes)>,
 <Function DEFAULT_ADMIN_ROLE()>,
 <Function accumulatedFees(address)>,
 <Function calculateTradeSourceAmount(address,address,tuple[])>,
 <Function calculateTradeTargetAmount(address,address,tuple[])>,
 <Function controllerType()>,
 <Function createPair(address,address)>,
 <Function createStrategy(address,address,tuple[2])>,
 <Function deleteStrategy(uint256)>,
 <Function getRoleAdmin(bytes32)>,
 <Function getRoleMember(bytes32,uint256)>,
 <Function getRoleMemberCount(bytes32)>,
 <Function grantRole(bytes32,address)>,
 <Function hasRole(bytes32,address)>,
 <Function initialize()>,
 <Function pair(address,address)>,
 <Function pairTradingFeePPM(address,address)>,
 <Function pairs()>,
 <Function pause()>,
 <Function paused()>,
 <Function postUpgrade(bytes)>,
 <Function renounceRole(bytes32,address)>,
 <Function revokeRole(bytes32,address)>,
 <Function roleAdmin(

In [44]:
carbon_controller.caller.strategiesByPair(
    manager.cfg.USDC_ADDRESS, manager.cfg.LINK_ADDRESS, 0, 0
)

[(8847341539944400050047739793225973497903,
  '0x245A0838F3ef2aB66BAb0D4a82eE17Eb0B4eA5BC',
  ['0x514910771AF9Ca656af840dff83E8264EcF986CA',
   '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'],
  [(0, 0, 0, 0), (0, 0, 0, 0)])]

## test_update_from_event_uniswap_v3

In [9]:
event = event_data['uniswap_v3_event']

assert event['args']['liquidity'] != [pool['liquidity'] for pool in manager.pool_data if pool['address'] == event['address']][0]

manager.update_from_event(event)

assert event['address'] in [pool['address'] for pool in manager.pool_data]
assert event['args']['liquidity'] == [pool['liquidity'] for pool in manager.pool_data if pool['address'] == event['address']][0]

## test_update_from_event_carbon_v1_update

In [10]:
event = event_data['carbon_v1_event_update']
assert event['args']['order0'][0] != [pool['y_0'] for pool in manager.pool_data if pool['cid'] == event['args']['id']][0]

manager.update_from_event(event)

assert event['args']['order0'][0] == [pool['y_0'] for pool in manager.pool_data if pool['cid'] == event['args']['id']][0]

## test_update_from_event_carbon_v1_create


In [11]:
event = event_data['carbon_v1_event_create']
manager.pool_data = [pool for pool in manager.pool_data if pool['cid'] != event['args']['id']]
assert event['args']['id'] not in [pool['cid'] for pool in manager.pool_data]

manager.update_from_event(event)

assert event['args']['id'] in [pool['cid'] for pool in manager.pool_data]

## test_update_from_event_carbon_v1_delete

In [12]:
event = event_data['carbon_v1_event_create']
manager.pool_data = [pool for pool in manager.pool_data if pool['cid'] != event['args']['id']]
assert event['args']['id'] not in [pool['cid'] for pool in manager.pool_data]

manager.update_from_event(event)

assert event['args']['id'] in [pool['cid'] for pool in manager.pool_data]

event['event'] = 'StrategyDeleted'

manager.update_from_event(event)

assert event['args']['id'] not in [pool['cid'] for pool in manager.pool_data]